In [ ]:
# Reference: https://doi.org/10.1103/PhysRevE.85.041502, although the boundaries here are free.

In [1]:
using OrdinaryDiffEq, ODEInterfaceDiffEq, Sundials, DiffEqDevTools

INFO: Recompiling stale cache file C:\Users\admin\.julia\lib\v0.6\DiffEqDiffTools.ji for module DiffEqDiffTools.
INFO: Recompiling stale cache file C:\Users\admin\.julia\lib\v0.6\OrdinaryDiffEq.ji for module OrdinaryDiffEq.


In [2]:
const T = Float64
abstract type AbstractFilamentCache end
abstract type AbstractMagneticForce end
abstract type AbstractInextensibilityCache end
abstract type AbstractSolver end
abstract type AbstractSolverCache end

In [3]:
struct FerromagneticContinuous <: AbstractMagneticForce
    ω :: T
    F :: Vector{T}
end

mutable struct FilamentCache{
        MagneticForce        <: AbstractMagneticForce,
        InextensibilityCache <: AbstractInextensibilityCache,
        SolverCache          <: AbstractSolverCache
            } <: AbstractFilamentCache
    N  :: Int
    μ  :: T
    Cm :: T
    x  :: SubArray{T,1,Vector{T},Tuple{StepRange{Int,Int}},true}
    y  :: SubArray{T,1,Vector{T},Tuple{StepRange{Int,Int}},true}
    z  :: SubArray{T,1,Vector{T},Tuple{StepRange{Int,Int}},true}
    A  :: Matrix{T}
    P  :: InextensibilityCache
    F  :: MagneticForce
    Sc :: SolverCache
end

In [4]:
struct NoHydroProjectionCache <: AbstractInextensibilityCache
    J         :: Matrix{T}
    P         :: Matrix{T}
    J_JT      :: Matrix{T}
    J_JT_LDLT :: Base.LinAlg.LDLt{T, SymTridiagonal{T}}
    P0        :: Matrix{T}

    NoHydroProjectionCache(N::Int) = new(
        zeros(N, 3*(N+1)),          # J
        zeros(3*(N+1), 3*(N+1)),    # P
        zeros(N,N),                 # J_JT
        Base.LinAlg.LDLt{T,SymTridiagonal{T}}(SymTridiagonal(zeros(N), zeros(N-1))),
        zeros(N, 3*(N+1))
    )
end

In [5]:
struct DiffEqSolverCache <: AbstractSolverCache
    S1 :: Vector{T}
    S2 :: Vector{T}

    DiffEqSolverCache(N::Integer) = new(zeros(T,3*(N+1)), zeros(T,3*(N+1)))
end

In [6]:
function FilamentCache(N=20; Cm=32, ω=200, Solver=SolverDiffEq)
    InextensibilityCache = NoHydroProjectionCache
    SolverCache = DiffEqSolverCache
    tmp = zeros(3*(N+1))
    FilamentCache{FerromagneticContinuous, InextensibilityCache, SolverCache}(
        N, N+1, Cm, view(tmp,1:3:3*(N+1)), view(tmp,2:3:3*(N+1)), view(tmp,3:3:3*(N+1)),
        zeros(3*(N+1), 3*(N+1)), # A
        InextensibilityCache(N), # P
        FerromagneticContinuous(ω, zeros(3*(N+1))),
        SolverCache(N)
    )
end

FilamentCache

In [7]:
function stiffness_matrix!(f::AbstractFilamentCache)
    N, μ, A = f.N, f.μ, f.A
    A[:] = eye(3*(N+1))
    for i in 1 : 3
        A[i,i] =    1
        A[i,3+i] = -2
        A[i,6+i] =  1

        A[3+i,i]   = -2
        A[3+i,3+i] =  5
        A[3+i,6+i] = -4
        A[3+i,9+i] =  1

        A[3*(N-1)+i,3*(N-3)+i] =  1
        A[3*(N-1)+i,3*(N-2)+i] = -4
        A[3*(N-1)+i,3*(N-1)+i] =  5
        A[3*(N-1)+i,3*N+i]     = -2

        A[3*N+i,3*(N-2)+i]     =  1
        A[3*N+i,3*(N-1)+i]     = -2
        A[3*N+i,3*N+i]         =  1

        for j in 2 : N-2
            A[3*j+i,3*j+i]     =  6
            A[3*j+i,3*(j-1)+i] = -4
            A[3*j+i,3*(j+1)+i] = -4
            A[3*j+i,3*(j-2)+i] =  1
            A[3*j+i,3*(j+2)+i] =  1
        end
    end
    scale!(A, -μ^4)
    nothing
end

stiffness_matrix! (generic function with 1 method)

In [8]:
function update_separate_coordinates!(f::AbstractFilamentCache, r)
    N, x, y, z = f.N, f.x, f.y, f.z
    @inbounds for i in 1 : length(x)
        x[i] = r[3*i-2]
        y[i] = r[3*i-1]
        z[i] = r[3*i]
    end
    nothing
end

function update_united_coordinates!(f::AbstractFilamentCache, r)
    N, x, y, z = f.N, f.x, f.y, f.z
    @inbounds for i in 1 : length(x)
        r[3*i-2] = x[i]
        r[3*i-1] = y[i]
        r[3*i]   = z[i]
    end
    nothing
end

function update_united_coordinates(f::AbstractFilamentCache)
    r = zeros(T, 3*length(f.x))
    update_united_coordinates!(f, r)
    r
end

update_united_coordinates (generic function with 1 method)

In [9]:
function initialize!(initial_conf_type::Symbol, f::AbstractFilamentCache)
    N, x, y, z = f.N, f.x, f.y, f.z
    if initial_conf_type == :StraightX
        x[:] = linspace(0, 1, N+1)
        y[:] = 0 .* x
        z[:] = 0 .* x
    else
        error("Unknown initial configuration requested.")
    end
    update_united_coordinates(f)
end

initialize! (generic function with 1 method)

In [10]:
function magnetic_force!(::FerromagneticContinuous, f::AbstractFilamentCache, t)
    # TODO: generalize this for different magnetic fields as well
    N, μ, Cm, ω, F = f.N, f.μ, f.Cm, f.F.ω, f.F.F
    F[1]         = -μ * Cm * cos(ω*t)
    F[2]         = -μ * Cm * sin(ω*t)
    F[3*(N+1)-2] =  μ * Cm * cos(ω*t)
    F[3*(N+1)-1] =  μ * Cm * sin(ω*t)
    nothing
end

magnetic_force! (generic function with 1 method)

In [11]:
struct SolverDiffEq <: AbstractSolver end

function (f::FilamentCache)(t, r, dr)
    @views f.x, f.y, f.z = r[1:3:end], r[2:3:end], r[3:3:end]
    jacobian!(f)
    projection!(f)
    magnetic_force!(f.F, f, t)
    A, P, F, S1, S2 = f.A, f.P.P, f.F.F, f.Sc.S1, f.Sc.S2

    # implement dr = P * (A*r + F) in an optimized way to avoid temporaries
    A_mul_B!(S1, A, r)
    S1 .+= F
    A_mul_B!(S2, P, S1)
    copy!(dr, S2)
    return dr
end

function (f::FilamentCache)(::Type{Val{:jac}}, t, r, J)
    A_mul_B!(J, f.P.P, f.A)
    nothing
end

function run(::SolverDiffEq; N=20, Cm=32, ω=200, time_end=1., solver=TRBDF2(autodiff=false), reltol=1e-6, abstol=1e-6)
    f = FilamentCache(N, Solver=SolverDiffEq, Cm=Cm, ω=ω)
    r0 = initialize!(:StraightX, f)
    stiffness_matrix!(FreeEnds(), f)
    prob = ODEProblem(f, r0, (0., time_end))
    sol = solve(prob, solver, dense=false, reltol=reltol, abstol=abstol)
end

run (generic function with 1 method)

In [12]:
function jacobian!(f::FilamentCache)
    N, x, y, z, J = f.N, f.x, f.y, f.z, f.P.J
    @inbounds for i in 1 : N
        J[i, 3*i-2]     = -2 * (x[i+1]-x[i])
        J[i, 3*i-1]     = -2 * (y[i+1]-y[i])
        J[i, 3*i]       = -2 * (z[i+1]-z[i])
        J[i, 3*(i+1)-2] =  2 * (x[i+1]-x[i])
        J[i, 3*(i+1)-1] =  2 * (y[i+1]-y[i])
        J[i, 3*(i+1)]   =  2 * (z[i+1]-z[i])
    end
    nothing
end

jacobian! (generic function with 1 method)

In [13]:
function projection!(f::FilamentCache)
    # implement P[:] = I - J'/(J*J')*J in an optimized way to avoid temporaries
    J, P, J_JT, J_JT_LDLT, P0 = f.P.J, f.P.P, f.P.J_JT, f.P.J_JT_LDLT, f.P.P0
    A_mul_Bt!(J_JT, J, J)
    LDLt_inplace!(J_JT_LDLT, J_JT)
    A_ldiv_B!(P0, J_JT_LDLT, J)
    At_mul_B!(P, P0, J)
    subtract_from_identity!(P)
    nothing
end

projection! (generic function with 1 method)

In [14]:
function subtract_from_identity!(A)
    scale!(-1, A)
    @inbounds for i in 1 : size(A,1)
        A[i,i] += 1
    end
    nothing
end

subtract_from_identity! (generic function with 1 method)

In [15]:
function LDLt_inplace!{T<:Real}(L::Base.LinAlg.LDLt{T,SymTridiagonal{T}}, A::Matrix{T})
    n = size(A,1)
    dv, ev = L.data.dv, L.data.ev
    @inbounds for (i,d) in enumerate(diagind(A))
        dv[i] = A[d]
    end
    @inbounds for (i,d) in enumerate(diagind(A,-1))
        ev[i] = A[d]
    end
    @inbounds @simd for i in 1 : n-1
        ev[i]   /= dv[i]
        dv[i+1] -= abs2(ev[i]) * dv[i]
    end
    L
end

LDLt_inplace! (generic function with 1 method)

In [16]:
N=20
abstols=1./10.^(3:6)
reltols=1./10.^(3:6)
f = FilamentCache(N, Solver=SolverDiffEq)
r0 = initialize!(:StraightX, f)
stiffness_matrix!(f)
prob = ODEProblem(f, r0, (0., 0.01))

setups = [
    # low accuracy
    #Dict(:alg => CVODE_BDF()),
    #Dict(:alg => Rosenbrock23(autodiff=false)),
    #Dict(:alg => Rodas4(autodiff=false)),
    #Dict(:alg => BS3()),
    #Dict(:alg => Tsit5()),
    #Dict(:alg => ImplicitEuler(autodiff=false)),
    #Dict(:alg => Trapezoid(autodiff=false)),
    #Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => radau()),
    #Dict(:alg => rodas()),
    #Dict(:alg => dop853()),

    # high accuracy
    """
    Dict(:alg => CVODE_BDF()),
    Dict(:alg => Vern7()),
    Dict(:alg => Vern9()),
    Dict(:alg => TRBDF2(autodiff=false)),
    Dict(:alg => Rodas4(autodiff=false)),
    Dict(:alg => Rodas5(autodiff=false)),
    Dict(:alg => ImplicitEuler(autodiff=false), :dts => 1e-6*ones(length(abstols)), :adaptive => false),
    Dict(:alg => radau()),
    Dict(:alg => dop853())
    """
];
names = [
    # low accuracy
    #"CVODE-BDF",
    #"Rosenbrock23",
    #"Rodas4",
    #"BS3",
    #"Tsit5",
    #"ImplicitEuler",
    #"Trapezoid",
    #"TRBDF2",
    "radau",
    #"rodas",
    #"dop853",

    # high accuracy
    """
    "CVODE_BDF",
    "Vern7",
    "Vern9",
    "TRBDF2",
    "Rodas4",
    "Rodas5",
    "ImplicitEuler_nonadaptive_1e-6",
    "radau",
    "dop853"
    """
];

In [17]:
sol = solve(prob, Vern9(), reltol=1e-14, abstol=1e-14)
test_sol = TestSolution(sol.t, sol.u)

retcode: Success
Interpolation: No interpolation
t: 9283-element Array{Float64,1}:
 0.0       
 5.57235e-7
 9.58262e-7
 1.49324e-6
 2.04886e-6
 2.69476e-6
 3.37975e-6
 4.17797e-6
 5.04105e-6
 6.0132e-6 
 7.20029e-6
 8.42041e-6
 9.78027e-6
 ⋮         
 0.0099892 
 0.00999022
 0.00999126
 0.00999224
 0.00999331
 0.00999434
 0.00999534
 0.00999637
 0.00999738
 0.00999844
 0.00999947
 0.01      
u: 9283-element Array{Array{Float64,1},1}:
 [0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.0, 0.0, 0.15  …  0.0, 0.9, 0.0, 0.0, 0.95, 0.0, 0.0, 1.0, 0.0, 0.0]                                                                 
 [3.59593e-15, -2.01704e-8, 0.0, 0.05, -1.27659e-9, 0.0, 0.1, 5.05306e-10, 0.0, 0.15  …  0.0, 0.9, -5.05306e-10, 0.0, 0.95, 1.27659e-9, 0.0, 1.0, 2.01704e-8, 0.0]          
 [2.84072e-14, -5.84698e-8, 0.0, 0.05, -5.74405e-9, 0.0, 0.1, 1.97338e-9, 0.0, 0.15  …  0.0, 0.9, -1.97338e-9, 0.0, 0.95, 5.74405e-9, 0.0, 1.0, 5.84698e-8, 0.0]            
 [1.49834e-13, -1.38789e-7, 0.0, 0.05, -1.8

In [18]:
# manually check that the solver actually works
sol2 = solve(prob, radau(), reltol=1e-5, abstol=1e-5)

retcode: Success
Interpolation: 1st order linear
t: 79-element Array{Float64,1}:
 0.0       
 1.0e-6    
 9.0e-6    
 7.3e-5    
 0.00030255
 0.00071217
 0.00106185
 0.0013415 
 0.00162116
 0.0018829 
 0.00214463
 0.00244662
 0.00261444
 ⋮         
 0.00896231
 0.00904039
 0.00911979
 0.00919919
 0.00932373
 0.00941903
 0.00951433
 0.00960061
 0.00968689
 0.00976516
 0.00984344
 0.01      
u: 79-element Array{Array{Float64,1},1}:
 [0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.1, 0.0, 0.0, 0.15  …  0.0, 0.9, 0.0, 0.0, 0.95, 0.0, 0.0, 1.0, 0.0, 0.0]                                                                                
 [-1.32836e-17, -6.37911e-8, 0.0, 0.05, -6.2712e-9, 0.0, 0.1, 2.24816e-9, 0.0, 0.15  …  0.0, 0.9, -2.24816e-9, 0.0, 0.95, 6.2712e-9, 0.0, 1.0, 6.37911e-8, 0.0]                            
 [-2.36733e-10, -4.3155e-6, 0.0, 0.05, -1.44866e-6, 0.0, 0.1, -1.12189e-7, 0.0, 0.15  …  0.0, 0.9, 1.12189e-7, 0.0, 0.95, 1.44866e-6, 0.0, 1.0, 4.3155e-6, 0.0]                            
 [

In [25]:
wp = WorkPrecisionSet(prob, abstols, reltols, setups; names=names, appxsol=test_sol, maxiters=Int(1e6))

LoadError: [91mMethodError: no method matching haskey(::String, ::Symbol)[0m
Closest candidates are:
  haskey([91m::Plots.Axis[39m, ::Symbol) at C:\Users\admin\.julia\v0.6\Plots\src\axes.jl:120
  haskey([91m::PyCall.PyObject[39m, ::Union{AbstractString, Symbol}) at C:\Users\admin\.julia\v0.6\PyCall\src\PyCall.jl:267
  haskey([91m::PyPlot.Figure[39m, ::Any) at C:\Users\admin\.julia\v0.6\PyPlot\src\PyPlot.jl:70
  ...[39m

In [24]:
using Plots; gr()

Plots.GRBackend()

In [21]:
plot(wp)

LoadError: [91mUndefVarError: wp not defined[39m